In [2]:
#%pip install yfinance
#pip install sdmx1



In [31]:
#$env:PYTHONPATH="src"      # PowerShell
import sys
import os

project_root = os.path.abspath("../src")
sys.path.append(project_root)

%load_ext autoreload
%autoreload
from curves.yield_curve import YieldCurve
from products.coupon_bond import CouponBond
from products.future import EquityFuture
from products.swap import InterestRateSwap
from products.option import EuropeanCall,Option,EuropeanPut
from models.black_scholes import BlackScholesModel
from models.calibration import ImpliedVolatilitySolver
from data_loader.yield_curve_loader import YieldCurveLoader
from data_loader.equity_loader import EquityLoader

from models.monte_carlo import MonteCarloPricer



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# les produits

In [ ]:
curve = YieldCurve(
    maturities=[1, 2, 3, 5],
    zero_rates=[0.02, 0.025, 0.03, 0.035]
)

print(curve.zero_rate(4))   # interpolation entre 3y et 5y
print(curve.discount_factor(4))
print(curve.zero_rate(0.5))



0.0325
0.8780954309205613
0.01


In [ ]:
bond = CouponBond(nominal=100, coupon_rate=0.05, maturity=5, payment_frequency=2)

print("Bond price =", bond.price(curve))
print("Bond price at t=1:", bond.price(curve,t=1))


Bond price = 106.98017032747649
Bond price at t=1: 109.0976296382205


In [ ]:
fut = EquityFuture(
    spot=4000,       # S&P 500 spot
    rate=0.03,       # 3%
    dividend_yield=0.015,  # 1.5%
    maturity=0.5     # 6 months
)

print("Future price =", fut.price())


Future price = 4030.1127817781353


In [ ]:
swap = InterestRateSwap(
    notional=1_000_000,
    fixed_rate=0.03,
    payment_times=[1, 2, 3]
)

print("Fixed Leg PV =", swap.pv_fixed_leg(curve))
print("Floating Leg PV =", swap.pv_floating_leg(curve))
print("Swap Value =", swap.price(curve))
print("Swap rate =", swap.swap_rate(curve))

print("\nFixed Leg PV =", swap.pv_fixed_leg(curve,t=1))
print("Floating Leg PV =", swap.pv_floating_leg(curve,t=1))
print("Swap Value =", swap.price(curve,t=1))
print("Swap rate =", swap.swap_rate(curve,t=1))

Fixed Leg PV = 85360.77849236093
Floating Leg PV = 66267.48803552707
Swap Value = -19093.290456833856
Swap rate = 0.02328967326889742

Fixed Leg PV = 87942.84293422407
Floating Leg PV = 48770.57549928599
Swap Value = -39172.26743493808
Swap rate = 0.017140392705175672


In [ ]:
call = EuropeanCall(strike=100, maturity=1.0)
model = BlackScholesModel(spot=105, rate=0.03,vol = 0.2)



print("Call price =", model.price(call))
print("Delta =", model.delta(call))
print("Gamma =", model.gamma(call))
print("Vega =", model.vega(call))


Call price = 12.638755916496294
Delta = 0.6893295440636719
Gamma = 0.01681549194942603
Vega = 37.078159748484396


In [ ]:
model2 = MonteCarloPricer(spot=105, rate=0.03, vol = 0.2)



print("Call price =", model2.price(call))
print("Delta =", model2.delta(call))
print("Gamma =", model2.gamma(call))
print("Vega =", model2.vega(call))

Call price = 12.525527486817198
Delta = 1902.5154354270855
Gamma = 86115997.25625342
Vega = 3525.7162977064827


In [ ]:
import streamlit as st

In [ ]:
# Market price of the call
market_price = 12.50

# Solver
solver = ImpliedVolatilitySolver(model)

iv = solver.implied_vol_call(call, market_price)

print("Implied volatility =", iv)

Implied volatility = 0.19625254622411945


# data

In [30]:
code = 'SR_2Y'
code = code.replace("SR_",'')
code = code.replace("Y",'')
int(code)

2

In [ ]:
YC = YieldCurveLoader()
df = YC.get_rate(key='SR_20Y')
#YC.reload_yield_curve()
df

FREQ  REF_AREA  CURRENCY  PROVIDER_FM  INSTRUMENT_FM  PROVIDER_FM_ID  DATA_TYPE_FM  TIME_PERIOD
B     U2        EUR       4F           G_N_A          SV_C_YM         SR_20Y        2025-11-24     3.298596
                                                                                    2025-11-25     3.263145
                                                                                    2025-11-26     3.277486
                                                                                    2025-11-27     3.273567
Name: value, dtype: float64

In [2]:
EQ = EquityLoader()
data = EQ.find('cac 40')
data.head()

,Symbole,Nom,Dernier cours,Secteur/Catégorie,Type,Change
0,^FCHI,CAC 40,"8 122,71",--,Indices,PAR
1,GC40.DE,AIS-AMUNDI CAC 40 ESG N,"142,96",--,Trackers,GER
2,CAC6L.PA,CAC 40 X6 Leverage NR,"3 102,46",--,Indices,PAR
3,LVC.PA,Amundi CAC 40 Daily (2x) Levera,"43,15",--,Trackers,PAR
4,CA10S.PA,CAC 40 X10 Short GR,"2 784,22",--,Indices,PAR


In [16]:
data = EQ.get_historic('MCP.XD',period='1y')
data.head()

e:\Cours3A\Modeles_de_pricing_des_derivatives\asset-pricing\src\data_loader\equity_loader.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, end=end, period=period)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,MCP.XD,MCP.XD,MCP.XD,MCP.XD,MCP.XD
Date,,,,,
2025-11-28,636.099976,637.450012,627.799988,628.799988,159833


In [17]:
import pandas as pd
data['date'] = pd.to_datetime(data.index)
data.head()

Price,Close,High,Low,Open,Volume,date
Ticker,MCP.XD,MCP.XD,MCP.XD,MCP.XD,MCP.XD,
Date,,,,,,
2025-11-28,636.099976,637.450012,627.799988,628.799988,159833,2025-11-28


In [20]:
data.columns = [ col[0] for col in data.columns.values]#'_'.join(col).strip()
data.head()

,Close,High,Low,Open,Volume,date
Date,,,,,,
2025-11-28,636.099976,637.450012,627.799988,628.799988,159833,2025-11-28


In [22]:
[ col for col in data.columns.values]

['Close', 'High', 'Low', 'Open', 'Volume', 'date']

In [ ]:
EQ = EquityLoader()
print(f' VIX : {EQ.get_VIX()}')
print(f' VCAC : {EQ.get_VCAC()}')

e:\Cours3A\Modèles de pricing des derivatives\asset-pricing\src\data_loader\equity_loader.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, end=end, period=period)
[*********************100%***********************]  1 of 1 completed
e:\Cours3A\Modèles de pricing des derivatives\asset-pricing\src\data_loader\equity_loader.py:96: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data.iloc[-1]['Close']) / 100


 VIX : 0.18559999465942384
 VCAC : 1.0000000000000003e-05


In [ ]:
EQ.get_implied_volatility('^FCHI')

np.float64(1.0000000000000003e-05)

In [33]:
EQ.get_implied_volatility('^FCHI')

np.float64(1.4980493847656249)

# Other

In [41]:
import yfinance as yf
from datetime import datetime
etf = yf.Ticker('EWQ')
options = etf.option_chain()
options.calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,EWQ251219C00032000,2025-08-18 15:33:26+00:00,32.0,11.70,9.50,13.7,0.0,0.0,1.0,1,1.498049,True,REGULAR,USD
1,EWQ251219C00040000,2025-11-21 17:21:41+00:00,40.0,3.70,0.00,0.0,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
2,EWQ251219C00042000,2025-05-13 15:06:21+00:00,42.0,2.40,0.75,5.0,0.0,0.0,1.0,0,0.907716,True,REGULAR,USD
3,EWQ251219C00043000,2025-10-08 13:30:04+00:00,43.0,2.45,0.55,3.0,0.0,0.0,1.0,7,0.533208,True,REGULAR,USD
4,EWQ251219C00044000,2025-09-12 16:49:03+00:00,44.0,0.88,0.05,3.8,0.0,0.0,NaN,5,0.871583,True,REGULAR,USD
